In [5]:
# ===================================================================
# SEL 6 (KODE LENGKAP & FINAL): JALANKAN ULANG ANALISIS UNTUK SEMUA FOLDER
# ===================================================================
# --- 1. PERSIAPAN DAN IMPORT ---
import matplotlib
# Baris ini SANGAT PENTING untuk mencegah Kernel Crash. HARUS paling atas.
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import cv2
import os
import random
import torch
import torchvision
from PIL import Image
from tqdm import tqdm
from torchvision.datasets import CocoDetection
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead

# --- 2. PUSAT KENDALI UTAMA ---
# Atur ke False untuk melewati bagian plotting yang berpotensi menyebabkan crash.
# Jika Anda sudah memperbaiki environment, Anda bisa mengubahnya menjadi True.
RUN_PLOTTING = False

# --- 3. KONFIGURASI GLOBAL ---
# Pastikan konfigurasi ini sesuai dengan proyek Anda.
try:
    import utils
    print("Helper 'utils.py' berhasil di-import.")
except ImportError:
    print("GAGAL meng-import 'utils'. Pastikan file tersebut ada.")
    # Definisikan fungsi dummy jika utils.py tidak ditemukan
    def collate_fn(batch): return tuple(zip(*batch))
    class utils: collate_fn = collate_fn

DATASET_ROOT = 'dataset'
CLASSES_TO_USE = [
    'baguette', 'cornbread', 'croissant', 'ensaymada', 'flatbread',
    'sourdough', 'wheat-bread', 'white-bread', 'whole-grain-bread', 'pandesal'
]
NUM_CLASSES = len(CLASSES_TO_USE)

# ===================================================================
# 4. FUNGSI UTAMA UNTUK ANALISIS ULANG
# ===================================================================
def rerun_analysis(experiment_dir, score_threshold=0.5):
    """
    Menjalankan ulang seluruh proses analisis (plotting, inferensi, metrik)
    untuk sebuah direktori eksperimen yang sudah ada.
    """
    print("="*80)
    print(f" Memulai analisis ulang untuk: {os.path.basename(experiment_dir)}")
    print(f" Dengan Score Threshold baru: {score_threshold}")
    print("="*80)

    best_model_path = os.path.join(experiment_dir, 'best_model.pth')
    log_path = os.path.join(experiment_dir, 'training_log.csv')

    # --- Bagian A: Plotting Kurva (Dilewati jika RUN_PLOTTING = False) ---
    if RUN_PLOTTING:
        print(f"\n>>> Membuat grafik dari: {log_path}...")
        if os.path.exists(log_path):
            try:
                log_df = pd.read_csv(log_path)
                fig, ax1 = plt.subplots(figsize=(14, 7))
                ax1.set_xlabel('Epoch'); ax1.set_ylabel('Rata-rata Training Loss', color='tab:red')
                ax1.plot(log_df['epoch'], log_df['loss'], 'o-', color='tab:red')
                ax1.tick_params(axis='y', labelcolor='tab:red'); ax1.grid(True, linestyle='--', alpha=0.6)
                ax2 = ax1.twinx()
                ax2.set_ylabel('Validation mAP (0.50:0.95)', color='tab:blue')
                ax2.plot(log_df['epoch'], log_df['mAP_0.50:0.95'], 's-', color='tab:blue')
                ax2.tick_params(axis='y', labelcolor='tab:blue')
                plt.title(f'Kurva Stabilitas Training: {os.path.basename(experiment_dir)}')
                plot_path = os.path.join(experiment_dir, 'training_stability_curve.png')
                plt.savefig(plot_path, dpi=200, bbox_inches='tight')
                plt.close(fig)
                print(f">>> Grafik disimpan di: {plot_path}")
            except Exception as e:
                print(f" GAGAL membuat grafik plot: {e}")
        else:
            print("File log tidak ditemukan.")
    else:
        print("\n>>> Pembuatan grafik dilewati sesuai konfigurasi.")

    # --- Bagian B: Analisis Mendalam (Model & Metrik) ---
    if not os.path.exists(best_model_path):
        print(f"\nANALISIS DILEWATI: Checkpoint 'best_model.pth' tidak ditemukan di folder ini.")
        return

    REPORT_DIR = os.path.join(experiment_dir, 'post_training_reports')
    INFERENCE_DIR = os.path.join(experiment_dir, 'inference_results')
    os.makedirs(REPORT_DIR, exist_ok=True); os.makedirs(INFERENCE_DIR, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # PERBAIKAN: Buat model dengan arsitektur yang sama persis seperti saat training
    print("Membuat kerangka model dari arsitektur pre-trained 'DEFAULT'...")
    model = ssdlite320_mobilenet_v3_large(weights='DEFAULT')
    
    # Ganti head-nya agar cocok dengan jumlah kelas Anda (10 kelas + background)
    num_anchors = model.anchor_generator.num_anchors_per_location()
    in_channels = [m[0][0].in_channels for m in model.head.classification_head.module_list]
    new_head = SSDLiteClassificationHead(in_channels=in_channels, num_anchors=num_anchors, num_classes=(NUM_CLASSES + 1), norm_layer=torch.nn.BatchNorm2d)
    model.head.classification_head = new_head
    
    # Setelah arsitektur benar, timpa dengan bobot hasil training Anda
    print(f"Memuat bobot hasil fine-tuning dari: {best_model_path}")
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.to(device); model.eval()
    print("Model berhasil dimuat dan siap untuk analisis.")
    
    # Definisikan dataset untuk analisis
    class AnalysisBreadDataset(CocoDetection):
        def __init__(self, root, annFile): super().__init__(root, annFile)
        def __getitem__(self, idx):
            img, target_list = super().__getitem__(idx)
            img_tensor = torchvision.transforms.functional.to_tensor(img)
            boxes, labels = [], []
            for ann in target_list:
                x, y, w, h = ann['bbox']
                if w > 0 and h > 0: boxes.append([x, y, x + w, y + h]); labels.append(ann['category_id'])
            target = {"boxes": torch.as_tensor(boxes, dtype=torch.float32), "labels": torch.as_tensor(labels, dtype=torch.int64), "image_id": torch.tensor([self.ids[idx]])}
            return img_tensor, target
            
    dataset_test = AnalysisBreadDataset(os.path.join(DATASET_ROOT, 'valid'), os.path.join(DATASET_ROOT, 'valid', 'filtered_annotations.coco.json'))
    data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)
    
    # Kalkulasi metrik
    per_class_counts = {'TP': np.zeros(NUM_CLASSES), 'FP': np.zeros(NUM_CLASSES), 'FN': np.zeros(NUM_CLASSES)}
    iou_thr = 0.5
    print(">>> Memulai kalkulasi metrik per kelas...")
    with torch.no_grad():
        for images, targets in tqdm(data_loader_test, desc="Mengevaluasi gambar"):
            outputs = model([img.to(device) for img in images])
            for target, output in zip(targets, outputs):
                gt_boxes, gt_labels = target['boxes'].numpy(), target['labels'].numpy()
                keep = output['scores'] > score_threshold
                pred_boxes, pred_labels = output['boxes'][keep].cpu().numpy(), output['labels'][keep].cpu().numpy()
                gt_matched = [False] * len(gt_boxes)
                for i in range(len(pred_boxes)):
                    best_iou, best_gt_idx = 0, -1
                    for j in range(len(gt_boxes)):
                        if gt_labels[j] == pred_labels[i]:
                            iou = torchvision.ops.box_iou(torch.from_numpy(pred_boxes[i:i+1]), torch.from_numpy(gt_boxes[j:j+1])).item()
                            if iou > best_iou: best_iou, best_gt_idx = iou, j
                    pred_class_idx = pred_labels[i] - 1
                    if 0 <= pred_class_idx < NUM_CLASSES:
                        if best_iou >= iou_thr and best_gt_idx != -1 and not gt_matched[best_gt_idx]:
                            per_class_counts['TP'][pred_class_idx] += 1; gt_matched[best_gt_idx] = True
                        else:
                            per_class_counts['FP'][pred_class_idx] += 1
                for j in range(len(gt_boxes)):
                    if not gt_matched[j]:
                        gt_class_idx = gt_labels[j] - 1
                        if 0 <= gt_class_idx < NUM_CLASSES: per_class_counts['FN'][gt_class_idx] += 1
    eps = 1e-12
    prec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FP'] + eps)
    rec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FN'] + eps)
    f1 = 2 * prec * rec / (prec + rec + eps)
    metrics_df = pd.DataFrame({'Kelas': CLASSES_TO_USE, 'TP': per_class_counts['TP'].astype(int), 'FP': per_class_counts['FP'].astype(int), 'FN': per_class_counts['FN'].astype(int), 'Precision': prec, 'Recall': rec, 'F1-Score': f1})
    metrics_csv_path = os.path.join(REPORT_DIR, "per_class_metrics.csv")
    metrics_df.to_csv(metrics_csv_path, index=False)
    print(f"\n>>> Metrik per kelas baru DISIMPAN di: {metrics_csv_path}")
    print(f"\nAnalisis ulang untuk {os.path.basename(experiment_dir)} SELESAI.\n")


# ===================================================================
# 5. KONTROL UTAMA & EKSEKUSI LOOP
# ===================================================================
BASE_OUTPUTS_DIR = 'outputs'
NEW_SCORE_THRESHOLD = 0.5

print("="*80)
print("MEMULAI PROSES ANALISIS ULANG UNTUK SEMUA FOLDER EKSPERIMEN")
print("="*80)

try:
    all_items = os.listdir(BASE_OUTPUTS_DIR)
    folders_to_process = [os.path.join(BASE_OUTPUTS_DIR, d) for d in all_items if os.path.isdir(os.path.join(BASE_OUTPUTS_DIR, d)) and d.startswith('ssdmobilenetv3_statis_baseline')]
    print(f"Ditemukan {len(folders_to_process)} folder eksperimen untuk diproses ulang.")
except FileNotFoundError:
    print(f"Direktori '{BASE_OUTPUTS_DIR}' tidak ditemukan. Pastikan nama folder sudah benar.")
    folders_to_process = []

if folders_to_process:
    for folder_path in sorted(folders_to_process):
        try:
            rerun_analysis(experiment_dir=folder_path, score_threshold=NEW_SCORE_THRESHOLD)
        except Exception as e:
            print(f"\n!!!! TERJADI ERROR FATAL saat memproses {folder_path} !!!!")
            print(f"Error: {e}\n")
            print("Melanjutkan ke folder berikutnya...")
            continue
    print("\n--- SEMUA PROSES ANALISIS ULANG TELAH SELESAI ---")
else:
    print("\nTidak ada folder untuk diproses.")

Helper 'utils.py' berhasil di-import.
MEMULAI PROSES ANALISIS ULANG UNTUK SEMUA FOLDER EKSPERIMEN
Ditemukan 17 folder eksperimen untuk diproses ulang.
 Memulai analisis ulang untuk: ssdmobilenetv3_statis_baseline_albuminatinon_test_e70_bs8_lr0_001
 Dengan Score Threshold baru: 0.5

>>> Pembuatan grafik dilewati sesuai konfigurasi.
Membuat kerangka model dari arsitektur pre-trained 'DEFAULT'...
Memuat bobot hasil fine-tuning dari: outputs\ssdmobilenetv3_statis_baseline_albuminatinon_test_e70_bs8_lr0_001\best_model.pth
Model berhasil dimuat dan siap untuk analisis.
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
>>> Memulai kalkulasi metrik per kelas...


Mengevaluasi gambar: 100%|██████████| 311/311 [00:10<00:00, 28.37it/s]



>>> Metrik per kelas baru DISIMPAN di: outputs\ssdmobilenetv3_statis_baseline_albuminatinon_test_e70_bs8_lr0_001\post_training_reports\per_class_metrics.csv

Analisis ulang untuk ssdmobilenetv3_statis_baseline_albuminatinon_test_e70_bs8_lr0_001 SELESAI.

 Memulai analisis ulang untuk: ssdmobilenetv3_statis_baseline_e100_bs16_lr0_001
 Dengan Score Threshold baru: 0.5

>>> Pembuatan grafik dilewati sesuai konfigurasi.
Membuat kerangka model dari arsitektur pre-trained 'DEFAULT'...
Memuat bobot hasil fine-tuning dari: outputs\ssdmobilenetv3_statis_baseline_e100_bs16_lr0_001\best_model.pth
Model berhasil dimuat dan siap untuk analisis.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
>>> Memulai kalkulasi metrik per kelas...


Mengevaluasi gambar: 100%|██████████| 311/311 [00:09<00:00, 32.88it/s]



>>> Metrik per kelas baru DISIMPAN di: outputs\ssdmobilenetv3_statis_baseline_e100_bs16_lr0_001\post_training_reports\per_class_metrics.csv

Analisis ulang untuk ssdmobilenetv3_statis_baseline_e100_bs16_lr0_001 SELESAI.

 Memulai analisis ulang untuk: ssdmobilenetv3_statis_baseline_e100_bs4_lr0_001
 Dengan Score Threshold baru: 0.5

>>> Pembuatan grafik dilewati sesuai konfigurasi.
Membuat kerangka model dari arsitektur pre-trained 'DEFAULT'...
Memuat bobot hasil fine-tuning dari: outputs\ssdmobilenetv3_statis_baseline_e100_bs4_lr0_001\best_model.pth
Model berhasil dimuat dan siap untuk analisis.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
>>> Memulai kalkulasi metrik per kelas...


...


>>> Metrik per kelas baru DISIMPAN di: outputs\ssdmobilenetv3_statis_baseline_e70_bs16_lr0_001\post_training_reports\per_class_metrics.csv

Analisis ulang untuk ssdmobilenetv3_statis_baseline_e70_bs16_lr0_001 SELESAI.

 Memulai analisis ulang untuk: ssdmobilenetv3_statis_baseline_e70_bs4_lr0_001
 Dengan Score Threshold baru: 0.5

>>> Pembuatan grafik dilewati sesuai konfigurasi.
Membuat kerangka model dari arsitektur pre-trained 'DEFAULT'...
Memuat bobot hasil fine-tuning dari: outputs\ssdmobilenetv3_statis_baseline_e70_bs4_lr0_001\best_model.pth
Model berhasil dimuat dan siap untuk analisis.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
>>> Memulai kalkulasi metrik per kelas...


Mengevaluasi gambar: 100%|██████████| 311/311 [00:08<00:00, 34.59it/s]



>>> Metrik per kelas baru DISIMPAN di: outputs\ssdmobilenetv3_statis_baseline_e70_bs4_lr0_001\post_training_reports\per_class_metrics.csv

Analisis ulang untuk ssdmobilenetv3_statis_baseline_e70_bs4_lr0_001 SELESAI.

 Memulai analisis ulang untuk: ssdmobilenetv3_statis_baseline_e70_bs8_lr0_001
 Dengan Score Threshold baru: 0.5

>>> Pembuatan grafik dilewati sesuai konfigurasi.
Membuat kerangka model dari arsitektur pre-trained 'DEFAULT'...
Memuat bobot hasil fine-tuning dari: outputs\ssdmobilenetv3_statis_baseline_e70_bs8_lr0_001\best_model.pth
Model berhasil dimuat dan siap untuk analisis.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
>>> Memulai kalkulasi metrik per kelas...


Mengevaluasi gambar: 100%|██████████| 311/311 [00:09<00:00, 34.03it/s]


>>> Metrik per kelas baru DISIMPAN di: outputs\ssdmobilenetv3_statis_baseline_e70_bs8_lr0_001\post_training_reports\per_class_metrics.csv

Analisis ulang untuk ssdmobilenetv3_statis_baseline_e70_bs8_lr0_001 SELESAI.


--- SEMUA PROSES ANALISIS ULANG TELAH SELESAI ---
